<a href="https://colab.research.google.com/github/NetraBatwe/Image-Colorisation-Project/blob/main/IC_Netra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torchvision
import torchvision.models as models
from torchvision import transforms,datasets
import numpy as np
from skimage.io import imread
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from torchsummary import summary
from skimage import io,color

In [ ]:
train_data = torchvision.datasets.Flowers102('data_dir',split="train", download=True, transform=True)
train_data = torchvision.datasets.Flowers102(root='./data',split="train", download=True, 
transform=transforms.Compose([transforms.ToTensor(),transforms.Resize((256,256))]))
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=60, shuffle=False)

train_data2 = torchvision.datasets.Flowers102(root='./data',split="train", download=True, 
transform=transforms.Compose([transforms.ToTensor(),transforms.Resize((256,256)),transforms.Grayscale()]))
train_dataloader2 = torch.utils.data.DataLoader(train_data2, batch_size=60, shuffle=False)

test_data = torchvision.datasets.Flowers102('data_dir',split="test", download=True, transform=True)
test_data = torchvision.datasets.Flowers102(root='./data',split="test", download=True, 
transform=transforms.Compose([transforms.ToTensor(),transforms.Resize((256,256))]))
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=60, shuffle=False)

test_data2= torchvision.datasets.Flowers102(root='./data',split="test", download=True, 
transform=transforms.Compose([transforms.ToTensor(),transforms.Resize((256,256)),transforms.Grayscale()]))
test_dataloader2= torch.utils.data.DataLoader(test_data2, batch_size=60, shuffle=False)

In [ ]:
import matplotlib.pyplot as plt
for data in train_dataloader:
    img, _ = data
    print(img.shape)
    break
for i, img in enumerate(img):
    plt.imshow(img.T)
    plt.show()
    break


In [ ]:
DEVICE = torch.device(f'cuda:{0}' if torch.cuda.is_available() else 'cpu')
print('Device:', DEVICE)

In [5]:
class ResNet(nn.Module):
  def __init__(self, input_size=128):
    super(ResNet, self).__init__()
   

    ## First half: ResNet
    resnet = models.resnet18(num_classes=64) 
    # Change first conv layer to accept single-channel (grayscale) input
    resnet.conv1.weight = nn.Parameter(resnet.conv1.weight.sum(dim=1).unsqueeze(1)) 
    # Extract midlevel features from ResNet-gray
    self.midlevel_resnet = nn.Sequential(*list(resnet.children())[0:6])

    ## Decoder
    self.decoder = nn.Sequential(     
      nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.Upsample(scale_factor=2),
      nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Upsample(scale_factor=2),
      nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.Conv2d(32, 2, kernel_size=3, stride=1, padding=1),
      nn.Upsample(scale_factor=2)
    )

  def forward(self, input):

    # Pass input through ResNet-gray to extract features
    midlevel_features = self.midlevel_resnet(input)

    # Upsample to get colors
    output = self.decoder(midlevel_features)
    return output


In [6]:
num_epochs = 15
batch_size = 60
model = ResNet().to(DEVICE)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_list=[]

In [7]:
def dataset_lab(cls, resize):
    def __getitem__(self, index):
        pil_image, target = cls.__getitem__(self, index)
        pil_image = resize(pil_image)
        lab_image = color.rgb2lab(pil_image)
        img = torch.from_numpy(lab_image).permute(2,0,1)
        return img[0].unsqueeze(0) / 100, img[1:] / 128, target
    return type(
        cls.__name__,
        (cls,),
        {
            "__getitem__": __getitem__,
        }
    )
print(type(img))

<class 'torch.Tensor'>


In [8]:
train_data=dataset_lab(torchvision.datasets.Flowers102,transforms.Resize((256,256)))
data= train_data("./data_dir")
train_labdataloader = torch.utils.data.DataLoader(data ,batch_size=60, shuffle=True)

In [ ]:
for epoch in range(1,20):
    # monitor training loss
    train_loss= 0.0    
    for data in train_dataloader2:
        img_gray,_=data
        img_gray=img_gray.to(DEVICE).float()
    for (img_l,img_ab,target) in (train_labdataloader):
        img_l = img_l.to(DEVICE)
        img_ab=img_ab.to(DEVICE).float()
        outputs = model(img_gray)  
        loss = criterion(outputs,img_ab)
        loss.backward()
        optimizer.step()
        train_loss+= loss.item()   
    train_loss= train_loss/len(train_labdataloader)

    print('Epoch: {} \tTraining Loss: {:.6f} '.format(epoch,train_loss))


In [ ]:
for (img_l,img_ab,target) in (train_labdataloader):
        img_l = img_l.to(DEVICE)
        img_ab=img_ab.to(DEVICE) 
        plt.imshow(img_l[0].reshape(256,256))
        print('l')
        plt.imshow(img_ab[0][0])
        break

In [ ]:
for i in range (0,30,30):
  plt.figure(figsize=(15, 3))
  for data in test_dataloader2:
        images, _ = data
        images = images.to(DEVICE)

  for i, img in enumerate(images.cpu()):
        if i >= 9: break
        plt.subplot(2, 9, i+1)
        img=img.resize(256,256)
        plt.imshow(img,cmap='gray')

  with torch.no_grad():
     outputs = model(images)
  images=images*100
  outputs=(outputs*128)
  lab_image=torch.cat((images,outputs),1)
  lab_image=lab_image.cpu().detach()
  rgb_image = lab2rgb(lab_image.permute(0,2,3,1))

  for data in test_dataloader:
       original_images, _ = data
       original_images = original_images.to(DEVICE)     
    
  for i, imgr in enumerate(rgb_image):
        if i >= 9: break
        plt.subplot(2, 9, 9+i+1) 
        plt.imshow(imgr)    